In [ ]:
import json
from shapely.geometry import shape
import geopandas as gpd
import matplotlib.pyplot as plt

def load_geojson(file_path):
    """
    Loads GeoJSON data from a file and converts it to a dictionary mapping postal codes to polygons.
    """
    with open(file_path, 'r') as file:
        geojson_data = json.load(file)

    postal_code_polygons = {feature['properties']['postal_code']: shape(feature['geometry']) for feature in geojson_data['features']}
    return postal_code_polygons

def plot_polygon(postal_code_polygons, postal_code):
    """
    Plots the polygon for a given postal code.
    """
    polygon = postal_code_polygons.get(postal_code)
    if polygon:
        gdf = gpd.GeoDataFrame(geometry=[polygon])
        gdf.plot()
        plt.title(f"Postal Code Polygon: {postal_code}")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.show()
    else:
        print(f"No polygon found for postal code: {postal_code}")

# Example usage
file_path = r'C:\Users\hamud\Documents\Retoflow\Project_Filter\Retoflow_Filter\Check_Out_Leon\Germany_geojson_data.geojson'
postal_code_polygons = load_geojson(file_path)

# Input the postal code you want to plot
input_postal_code = input("Enter a postal code: ")

# Plot the polygon for the specified postal code
plot_polygon(postal_code_polygons, input_postal_code)



In [4]:
from pymongo import MongoClient
from shapely.geometry import shape

class MongoDBDataRetriever:
    def __init__(self, database_name, collection_name):
        self.client = MongoClient('localhost', 27017)
        self.db = self.client[database_name]
        self.collection = self.db[collection_name]

    def retrieve_geojson_data(self):
        """
        Retrieves GeoJSON data from MongoDB and converts it to a dictionary mapping postal codes to polygons.
        """
        cursor = self.collection.find({}, {'_id': 0, 'postal_code': 1, 'geometry': 1})  # Include only necessary fields
        geojson_data = {'features': list(cursor)}
        postal_code_polygons = {feature['postal_code']: shape(feature['geometry']) for feature in geojson_data['features']}
        return postal_code_polygons

    def close_connection(self):
        self.client.close()
        print("Closed MongoDB connection.")


